In [1]:
import pandas as pd
import seaborn as sn
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

In [14]:
data = pd.read_csv('sonar.all-data',header=None)
X = data[data.columns[0:60]]
Y = data[data.columns[60]]
print(data.shape)
print(data.head())
#data.hist()
#plt.show()
print(Y.shape)
#data.describe()

(208, 61)
       0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9   ...      51      52      53      54      55      56      57  \
0  0.2111  ...  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180  0.0084   
1  0.2872  ...  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140  0.0049   
2  0.6194  ...  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316  0.0164   
3  0.1264  ...  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050  0.0044   
4  0.4459  ...  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072  0.0048   

       58      59  60  
0  0.0090  0.0032   R  
1  0.0052  0.0044   R  
2  0.0095 

In [3]:
#scatter_matrix(data)
#plt.show()

In [4]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels),labels] = 1
    return one_hot_encode


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
X,Y = shuffle(X,Y,random_state=1 )
encoder = LabelEncoder()
encoder.fit(Y)

Y = encoder.transform(Y)
Y= one_hot_encode(Y)


X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=1)


In [11]:
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
learning_rate = 0.3
epoch = 1000
cost_history = np.empty(shape=[1],dtype=float)
n_dim = X.shape[1]
n_class = 20
model_path = "{}/model".format(os.getcwd())
n_hidden_1 = 60
n_hidden_2 = 60
n_hidden_3 = 60
n_hidden_4 = 60
x = tf.placeholder(tf.float32,[None,n_dim])
W = tf.Variable(tf.zeros([n_dim,n_class]))
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.placeholder(tf.float32,[None,n_class])


In [12]:
def multilayer_perceptron(x,weights,biases):
    layer_1 = tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    
    layer_3 = tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
    
    layer_4 = tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    
    out_layer = tf.matmul(layer_4,weights['out'])+biases['out']
    return out_layer


In [13]:
print(y_)
weights = {
    'h1':tf.Variable(tf.truncated_normal([n_dim,n_hidden_1])),
    'h2':tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2])),
    'h3':tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3])),
    'h4':tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4])),
    'out':tf.Variable(tf.truncated_normal([n_hidden_4,n_class])),
}

biases = {
    'b1':tf.Variable(tf.truncated_normal([n_hidden_1])),
    'b2':tf.Variable(tf.truncated_normal([n_hidden_2])),
    'b3':tf.Variable(tf.truncated_normal([n_hidden_3])),
    'b4':tf.Variable(tf.truncated_normal([n_hidden_4])),
    'out':tf.Variable(tf.truncated_normal([n_class])),
}

init = tf.global_variables_initializer()
saver = tf.train.Saver()

y = multilayer_perceptron(x,weights,biases)

cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y,labels = y_))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)
sess = tf.Session()
sess.run(init)

mse_history = []
accuracy_history = []
for epch in range(epoch):
    sess.run(training_step,feed_dict = {x:X_train,y_:y_train})
    cost = sess.run(cost_function,feed_dict = {x:X_train,y_:y_train})
    cost_history = np.append(cost_history,cost)
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    pred_y = sess.run(y,feed_dict = {x:X_test})
    mse = tf.reduce_mean(tf.square(pred_y-y_test))
    mse_ = sess.run(mse)
    mse_history.append(mse_)
    accuracy = sess.run(accuracy,feed_dict={x:X_train,y_:y_train})
    accuracy_history.append(accuracy)
    print("Epoch : {}, Cost : {}, MSE : {}, Train_Accuracy : {}".format(epch,cost,mse_,accuracy))
    if(accuracy>98):
        break
save_path = saver.save(sess,model_path)
print("Model saved at : {}".format(save_path))


Tensor("Placeholder_3:0", shape=(?, 20), dtype=float32)
Epoch : 0, Cost : 40.48017120361328, MSE : 1951.6847703607966, Train_Accuracy : 0.04836732894182205
Epoch : 1, Cost : 66.25694274902344, MSE : 2210.1696026657537, Train_Accuracy : 0.1977434903383255
Epoch : 2, Cost : 39.127201080322266, MSE : 567.0931149079453, Train_Accuracy : 0.1610044240951538
Epoch : 3, Cost : 23.244112014770508, MSE : 158.42868176144393, Train_Accuracy : 0.10933718830347061
Epoch : 4, Cost : 7.3778557777404785, MSE : 10.882234734611576, Train_Accuracy : 0.02913353592157364
Epoch : 5, Cost : 3.2459664344787598, MSE : 0.8040119276221224, Train_Accuracy : 0.1610044240951538
Epoch : 6, Cost : 3.2274563312530518, MSE : 0.7848584727607466, Train_Accuracy : 0.1610044240951538
Epoch : 7, Cost : 3.2095284461975098, MSE : 0.7665183505679536, Train_Accuracy : 0.1610044240951538
Epoch : 8, Cost : 3.1921563148498535, MSE : 0.7489550426803354, Train_Accuracy : 0.1610044240951538
Epoch : 9, Cost : 3.1753408908843994, MSE : 

Epoch : 81, Cost : 2.6946115493774414, MSE : 0.39104569144375145, Train_Accuracy : 0.19771206378936768
Epoch : 82, Cost : 2.692391872406006, MSE : 0.3915854245633976, Train_Accuracy : 0.19771206378936768
Epoch : 83, Cost : 2.6902341842651367, MSE : 0.39218297877637337, Train_Accuracy : 0.19771206378936768
Epoch : 84, Cost : 2.688145160675049, MSE : 0.3928356402034894, Train_Accuracy : 0.19771206378936768
Epoch : 85, Cost : 2.686099052429199, MSE : 0.3935410964058154, Train_Accuracy : 0.19771206378936768
Epoch : 86, Cost : 2.6841182708740234, MSE : 0.3942975659788209, Train_Accuracy : 0.19771206378936768
Epoch : 87, Cost : 2.68217396736145, MSE : 0.39510297865773425, Train_Accuracy : 0.19771206378936768
Epoch : 88, Cost : 2.680285692214966, MSE : 0.3959546108291095, Train_Accuracy : 0.19771206378936768
Epoch : 89, Cost : 2.6784634590148926, MSE : 0.39685129040473044, Train_Accuracy : 0.19771206378936768
Epoch : 90, Cost : 2.6766769886016846, MSE : 0.3977906808950184, Train_Accuracy : 0.

Epoch : 161, Cost : 2.6211471557617188, MSE : 0.4985133068636552, Train_Accuracy : 0.19771206378936768
Epoch : 162, Cost : 2.6208770275115967, MSE : 0.4998567061873736, Train_Accuracy : 0.19771206378936768
Epoch : 163, Cost : 2.6206042766571045, MSE : 0.5011918228567765, Train_Accuracy : 0.19771206378936768
Epoch : 164, Cost : 2.6203370094299316, MSE : 0.5025186559423362, Train_Accuracy : 0.19771206378936768
Epoch : 165, Cost : 2.6200742721557617, MSE : 0.5038366086402203, Train_Accuracy : 0.19771206378936768
Epoch : 166, Cost : 2.6198325157165527, MSE : 0.5051462996654449, Train_Accuracy : 0.19771206378936768
Epoch : 167, Cost : 2.6195929050445557, MSE : 0.506447354555771, Train_Accuracy : 0.19771206378936768
Epoch : 168, Cost : 2.6193525791168213, MSE : 0.5077400008402788, Train_Accuracy : 0.19771206378936768
Epoch : 169, Cost : 2.619105815887451, MSE : 0.5090242166546166, Train_Accuracy : 0.19771206378936768
Epoch : 170, Cost : 2.618889570236206, MSE : 0.510299683612447, Train_Accur

Epoch : 242, Cost : 2.610307455062866, MSE : 0.5813035756512362, Train_Accuracy : 0.19771206378936768
Epoch : 243, Cost : 2.610243797302246, MSE : 0.5820405120770974, Train_Accuracy : 0.19771206378936768
Epoch : 244, Cost : 2.6101958751678467, MSE : 0.5827718023907452, Train_Accuracy : 0.19771206378936768
Epoch : 245, Cost : 2.6101479530334473, MSE : 0.5834976961040278, Train_Accuracy : 0.19771206378936768
Epoch : 246, Cost : 2.610090494155884, MSE : 0.5842179200266852, Train_Accuracy : 0.19771206378936768
Epoch : 247, Cost : 2.610034465789795, MSE : 0.5849327966099049, Train_Accuracy : 0.19771206378936768
Epoch : 248, Cost : 2.609981060028076, MSE : 0.5856420964503832, Train_Accuracy : 0.19771206378936768
Epoch : 249, Cost : 2.609933376312256, MSE : 0.5863462238390397, Train_Accuracy : 0.19771206378936768
Epoch : 250, Cost : 2.6098837852478027, MSE : 0.587044989228223, Train_Accuracy : 0.19771206378936768
Epoch : 251, Cost : 2.609830617904663, MSE : 0.5877384109914813, Train_Accuracy 

Epoch : 322, Cost : 2.60774564743042, MSE : 0.6259025668899567, Train_Accuracy : 0.19771206378936768
Epoch : 323, Cost : 2.607740640640259, MSE : 0.6263133022313812, Train_Accuracy : 0.19771206378936768
Epoch : 324, Cost : 2.6077229976654053, MSE : 0.6267212148039983, Train_Accuracy : 0.19771206378936768
Epoch : 325, Cost : 2.6077067852020264, MSE : 0.6271263360576709, Train_Accuracy : 0.19771206378936768
Epoch : 326, Cost : 2.6076860427856445, MSE : 0.6275286103396137, Train_Accuracy : 0.19771206378936768
Epoch : 327, Cost : 2.6076738834381104, MSE : 0.6279281294964435, Train_Accuracy : 0.19771206378936768
Epoch : 328, Cost : 2.607649564743042, MSE : 0.6283249067536102, Train_Accuracy : 0.19771206378936768
Epoch : 329, Cost : 2.6076364517211914, MSE : 0.6287187849110837, Train_Accuracy : 0.19771206378936768
Epoch : 330, Cost : 2.607619524002075, MSE : 0.6291100936639771, Train_Accuracy : 0.19771206378936768
Epoch : 331, Cost : 2.6076135635375977, MSE : 0.6294987606645933, Train_Accura

Epoch : 403, Cost : 2.606928586959839, MSE : 0.651625739524812, Train_Accuracy : 0.19771206378936768
Epoch : 404, Cost : 2.6069228649139404, MSE : 0.651865824129099, Train_Accuracy : 0.19771206378936768
Epoch : 405, Cost : 2.6069250106811523, MSE : 0.6521044887487604, Train_Accuracy : 0.19771206378936768
Epoch : 406, Cost : 2.606920003890991, MSE : 0.652341622980706, Train_Accuracy : 0.19771206378936768
Epoch : 407, Cost : 2.6068978309631348, MSE : 0.6525772732998021, Train_Accuracy : 0.19771206378936768
Epoch : 408, Cost : 2.6068975925445557, MSE : 0.6528114163391081, Train_Accuracy : 0.19771206378936768
Epoch : 409, Cost : 2.6068966388702393, MSE : 0.6530441001602842, Train_Accuracy : 0.19771206378936768
Epoch : 410, Cost : 2.606893539428711, MSE : 0.6532752729306845, Train_Accuracy : 0.19771206378936768
Epoch : 411, Cost : 2.606884002685547, MSE : 0.6535050699369218, Train_Accuracy : 0.19771206378936768
Epoch : 412, Cost : 2.606889486312866, MSE : 0.6537334844542128, Train_Accuracy 

Epoch : 484, Cost : 2.6066324710845947, MSE : 0.6669905591889791, Train_Accuracy : 0.19771206378936768
Epoch : 485, Cost : 2.6066224575042725, MSE : 0.6671375206250594, Train_Accuracy : 0.19771206378936768
Epoch : 486, Cost : 2.6066229343414307, MSE : 0.6672836751544023, Train_Accuracy : 0.19771206378936768
Epoch : 487, Cost : 2.6066174507141113, MSE : 0.6674289154655036, Train_Accuracy : 0.19771206378936768
Epoch : 488, Cost : 2.6066205501556396, MSE : 0.6675733541689508, Train_Accuracy : 0.19771206378936768
Epoch : 489, Cost : 2.6066126823425293, MSE : 0.6677169374876805, Train_Accuracy : 0.19771206378936768
Epoch : 490, Cost : 2.6066172122955322, MSE : 0.6678596932277134, Train_Accuracy : 0.19771206378936768
Epoch : 491, Cost : 2.6066133975982666, MSE : 0.6680016389542773, Train_Accuracy : 0.19771206378936768
Epoch : 492, Cost : 2.6066172122955322, MSE : 0.668142737305026, Train_Accuracy : 0.19771206378936768
Epoch : 493, Cost : 2.606623411178589, MSE : 0.6682830680518587, Train_Acc

Epoch : 565, Cost : 2.6064999103546143, MSE : 0.6765607910871, Train_Accuracy : 0.19771206378936768
Epoch : 566, Cost : 2.606496810913086, MSE : 0.6766541556339848, Train_Accuracy : 0.19771206378936768
Epoch : 567, Cost : 2.6065032482147217, MSE : 0.6767470400050407, Train_Accuracy : 0.19771206378936768
Epoch : 568, Cost : 2.6065003871917725, MSE : 0.6768394408947365, Train_Accuracy : 0.19771206378936768
Epoch : 569, Cost : 2.6065011024475098, MSE : 0.6769313333700218, Train_Accuracy : 0.19771206378936768
Epoch : 570, Cost : 2.6065046787261963, MSE : 0.677022728858975, Train_Accuracy : 0.19771206378936768
Epoch : 571, Cost : 2.606506824493408, MSE : 0.6771136063932931, Train_Accuracy : 0.19771206378936768
Epoch : 572, Cost : 2.60650634765625, MSE : 0.6772040242896524, Train_Accuracy : 0.19771206378936768
Epoch : 573, Cost : 2.606508493423462, MSE : 0.6772939446652211, Train_Accuracy : 0.19771206378936768
Epoch : 574, Cost : 2.606497287750244, MSE : 0.6773834228706397, Train_Accuracy : 

KeyboardInterrupt: 

In [ ]:
plt.plot(mse_history,'r')
plt.show()
plt.plot(accuracy_history)
plt.show()

correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print("Test Accuracy : {}".format(sess.run(accuracy,feed_dict={x:X_test,y_:y_test})))

pred_y = sess.run(y,feed_dict={x:X_test})
mse = tf.reduce_mean(tf.square(pred_y-y_test))
print("MSE : ",format(round(sess.run(mse),4)))